# GHZ state

The purpose of this notebook is to simulate the GHZ experiment
described in the IBM Quantum Experience tutorial in the section
entitled 

    Multiple Qubits, Gates, and Entangled States/GHZ states
    
If you understand our "Bell_and_CHSH_inequalities" notebook,
this notebook uses very similar math.

It uses the following results whose proofs use techniques already covered 
in our "Bell_and_CHSH_inequalities" notebook

$\langle b_X| = \langle b_Z| H$

$\langle b_Y| = \langle b_Z| H S^\dagger$

for $b=0, 1$.

$\langle\psi| \sigma_A(0) \sigma_B(1) \sigma(2)|\psi\rangle = \sum_{b_0 + b_1 + b_2 = 0, 2} Prob(b_0, b_1, b_2) - \sum_{b_0 + b_1 + b_2 = 1, 3} Prob(b_0, b_1, b_2)$

In [1]:
from SEO_writer import *
from SEO_simulator import *
import numpy as np

In [2]:
def write_ghz_plus(file_prefix, ghz_only=True, meas=None):
    num_bits = 3
    z_axis = 3
    emb = CktEmbedder(num_bits, num_bits)
    print('-------------------', file_prefix)
    wr = SEO_writer(file_prefix, emb)
    wr.write_one_bit_gate(0, OneBitGates.had2)
    wr.write_one_bit_gate(1, OneBitGates.had2)
    wr.write_one_bit_gate(2, OneBitGates.sigx)

    control_pos = 0
    target_pos = 2
    trols = Controls.new_knob(num_bits, control_pos, kind=True)
    wr.write_controlled_one_bit_gate(
        target_pos, trols, OneBitGates.sigx)

    control_pos = 1
    target_pos = 2
    trols = Controls.new_knob(num_bits, control_pos, kind=True)
    wr.write_controlled_one_bit_gate(
        target_pos, trols, OneBitGates.sigx)

    wr.write_one_bit_gate(0, OneBitGates.had2)
    wr.write_one_bit_gate(1, OneBitGates.had2)
    wr.write_one_bit_gate(2, OneBitGates.had2)

    if not ghz_only:
        for pos in range(3):
            m = meas[pos]
            if m == 1:
                wr.write_one_bit_gate(pos, OneBitGates.had2)
            elif m == 2:
                wr.write_one_bit_gate(pos,
                    OneBitGates.rot_ax, [np.pi/4, z_axis])  # S^\dagger(pos)
                wr.write_one_bit_gate(pos, OneBitGates.had2)
            else:
                assert False
    wr.close_files()
    pic_file = file_prefix + '_' + str(num_bits) + '_ZLpic.txt'
    with open(pic_file) as f:
        print(f.read())
    init_st_vec = SEO_simulator.get_standard_basis_st([0, 0, 0])
    sim = SEO_simulator(file_prefix, num_bits, init_st_vec)
    sim.describe_fin_st(print_st_vec=True, do_pp=True,
                        omit_zero_amps=True)
    fin_st_vec = sim.cur_st_vec_list[0]
    print('Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:')
    prob_arr = np.abs(fin_st_vec)**2
    print(prob_arr)
    mean = prob_arr[0, 0, 0]  \
            + prob_arr[0, 1, 1] \
            + prob_arr[1, 0, 1] \
            + prob_arr[1, 1, 0] \
            - prob_arr[1, 1, 1] \
            - prob_arr[0, 0, 1] \
            - prob_arr[1, 0, 0] \
            - prob_arr[0, 1, 0]
    print('mean=', mean)
    return mean


In [3]:
# sigz(0)sigz(1)sigz(2) measurement
file_prefix = 'io_folder/ghz_zzz_meas'
mean_zzz = write_ghz_plus(file_prefix, ghz_only=True)

------------------- io_folder/ghz_zzz_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   

Number of lines in file = 8

Number of Elem. Ops = 8

final state vector
(zero bit first in state tuple)
(0, 0, 0) (0.707106781187+0j)
(1, 1, 1) (-0.707106781187+0j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (0.5, 0.5), 1: (0.5, 0.5), 2: (0.5, 0.5)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  5.00000000e-01   4.53552674e-35]
  [  1.93646401e-37   1.84432228e-70]]

 [[  1.84432228e-70   1.93646401e-37]
  [  4.53552674e-35   5.00000000e-01]]]
mean= -4.55489137516e-35


In [4]:
# sigy(0)sigy(1)sigx(2) measurement
file_prefix = 'io_folder/ghz_yyx_meas'
mean_yyx = write_ghz_plus(file_prefix, ghz_only=False, meas=[2, 2, 1])


------------------- io_folder/ghz_yyx_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   Rz  
|   |   H   
|   Rz  |   
|   H   |   
H   |   |   

Number of lines in file = 13

Number of Elem. Ops = 13

final state vector
(zero bit first in state tuple)
(0, 0, 0) (-9.10939765373e-35+0.5j)
(0, 1, 1) (2.52852606327e-33+0.5j)
(1, 0, 1) (-9.10939765373e-35+0.5j)
(1, 1, 0) (2.52852606327e-33+0.5j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (0.5, 0.5), 1: (0.5, 0.5), 2: (0.5, 0.5)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  2.50000000e-01   7.00507475e-33]
  [  6.90143623e-33   2.50000000e-01]]

 [[  7.00507475e-33   2.50000000e-01]
  [  2.50000000e-01   6.90143623e-33]]]
mean= 1.0


In [5]:
# sigy(0)sigx(1)sigy(2) measurement
file_prefix = 'io_folder/ghz_yxy_meas'
mean_yxy = write_ghz_plus(file_prefix, ghz_only=False, meas=[2, 1, 2])

------------------- io_folder/ghz_yxy_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   Rz  
|   |   H   
|   H   |   
Rz  |   |   
H   |   |   

Number of lines in file = 13

Number of Elem. Ops = 13

final state vector
(zero bit first in state tuple)
(0, 0, 0) (1.96261557335e-17+0.5j)
(0, 1, 1) 0.5j
(1, 0, 1) 0.5j
(1, 1, 0) (1.96261557335e-17+0.5j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (0.5, 0.5), 1: (0.5, 0.5), 2: (0.49999999999999994, 0.5)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  2.50000000e-01   4.67780938e-33]
  [  6.18278669e-33   2.50000000e-01]]

 [[  6.18278669e-33   2.50000000e-01]
  [  2.50000000e-01   4.67780938e-33]]]
mean= 1.0


In [6]:
# sigx(0)sigy(1)sigy(2) measurement
file_prefix = 'io_folder/ghz_xyy_meas'
mean_xyy = write_ghz_plus(file_prefix, ghz_only=False, meas=[1, 2, 2])

------------------- io_folder/ghz_xyy_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   H   
|   Rz  |   
|   H   |   
Rz  |   |   
H   |   |   

Number of lines in file = 13

Number of Elem. Ops = 13

final state vector
(zero bit first in state tuple)
(0, 0, 0) (1.96261557335e-17+0.5j)
(0, 1, 1) 0.5j
(1, 0, 1) 0.5j
(1, 1, 0) (1.96261557335e-17+0.5j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (0.5, 0.5), 1: (0.5, 0.5), 2: (0.49999999999999994, 0.5)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  2.50000000e-01   4.67780938e-33]
  [  6.18278669e-33   2.50000000e-01]]

 [[  6.18278669e-33   2.50000000e-01]
  [  2.50000000e-01   4.67780938e-33]]]
mean= 1.0


In [7]:
# sigx(0)sigx(1)sigx(2) measurement
file_prefix = 'io_folder/ghz_xxx_meas'
mean_xxx = write_ghz_plus(file_prefix, ghz_only=False, meas=[1, 1, 1])

------------------- io_folder/ghz_xxx_meas
|   |   H   
|   H   |   
X   |   |   
X---+---@   
X---@   |   
|   |   H   
|   H   |   
H   |   |   
|   |   H   
|   H   |   
H   |   |   

Number of lines in file = 11

Number of Elem. Ops = 11

final state vector
(zero bit first in state tuple)
(0, 0, 1) (0.5+0j)
(0, 1, 0) (0.5+0j)
(1, 0, 0) (0.5+0j)
(1, 1, 1) (0.5+0j)
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (0.5, 0.5), 1: (0.5, 0.5), 2: (0.5, 0.5)}
Prob(bit0=i, bit1=j, bit2=k) for i,j,k,=0,1:
[[[  1.98108637e-35   2.50000000e-01]
  [  2.50000000e-01   1.98108637e-35]]

 [[  2.50000000e-01   1.98108637e-35]
  [  1.98108637e-35   2.50000000e-01]]]
mean= -1.0


Let

$mean\_abc = \langle\psi| \sigma_A(0) \sigma_B(1)\sigma_C(2)|\psi\rangle$

where

$|\psi\rangle = \frac{1}{\sqrt{2}}(|000\rangle + |111\rangle)$

We expect

$mean\_yyx = mean\_yxy = mean\_xyy = 1$

and 

$mean\_xxx=-1$.

This could never happen classically.

In [8]:
print('-----------------------')
print('mean_yyx =', mean_yyx)
print('mean_yxy =', mean_yxy)
print('mean_xyy =', mean_xyy)
print('mean_xxx =', mean_xxx)

-----------------------
mean_yyx = 1.0
mean_yxy = 1.0
mean_xyy = 1.0
mean_xxx = -1.0
